In [1]:
import os
import cv2
import time
import numpy as np
import numpy as np
import os
import cv2
import keras
import sklearn
import pandas
from time import time
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.utils import to_categorical
from keras.models import load_model
from keras.layers import *
from keras import layers
from keras import Model
from keras.callbacks import TensorBoard
from keras import optimizers
import matplotlib.pyplot as plt
from keras.applications import *
from sklearn.metrics import classification_report
import time
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from keras.applications.resnet50 import preprocess_input
from keras.applications import *
from keras import regularizers

In [94]:
v_path = r"F:\violent_video_tagging\violence-video-classification\data\single_video_frames\violent\\"
nv_path = r"F:\violent_video_tagging\violence-video-classification\data\single_video_frames\non_violent\\"

# v_path = r"F:\violent_video_tagging\violence-video-classification\data\final\train\violent\\"
# nv_path = r"F:\violent_video_tagging\violence-video-classification\data\final\train\non_violent\\"

In [103]:
violent_frames = []
non_violent_frames = []

for frame in os.listdir(v_path):
    frame = cv2.imread(os.path.join(v_path,frame))
    violent_frames.append(frame)


for frame in os.listdir(nv_path):
    frame = cv2.imread(os.path.join(nv_path,frame))
    non_violent_frames.append(frame)

In [104]:
#Creates sequence 
# SHAPE : (N_SAMPLES,TIMESTEP,WIDTH,HEIGHT,CHANNEL)
def create_seq(violent_frames, non_violent_frames):
    
    print("+++ Creating Sequence... +++")
    
    violent_vid = []
    non_violent_vid = []

    i = 0
    while i < len(violent_frames):
        violent_vid.append(violent_frames[i:i+30])
        i = i+30


    i = 0
    while i < len(non_violent_frames):
        non_violent_vid.append(non_violent_frames[i:i+30])
        i = i+30
    try:   
        violent_vid = np.asarray(violent_vid)
    except:
        violent_vid = violent_vid.reshape(0,0,0,0,0)
    try:
        non_violent_vid = np.asarray(non_violent_vid)
    except:
        non_violent_vid = non_violent_vid.reshape(0,0,0,0,0)
    
    return violent_vid, non_violent_vid


In [105]:
def result(x,y):
    
    print("+++ Generating result... +++")
    
    pred = model.predict(x)
#     print('First prediction:', pred)
    
    score = model.evaluate(x, y,verbose=1)
    print("-----------------------------")
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    
    prediction = []
    for p in pred:
        if p>=.5:
            prediction.append(1)
        else:
            prediction.append(0)
    print("-----------------------------")
    print("Classification report")
    print("-----------------------------")
    print(classification_report(y, prediction))
    
    print("-----------------------------")
    print("Confusion Matrix")
    print("-----------------------------")
    conf_mat = confusion_matrix(y, prediction)
    print(conf_mat)
    

In [106]:
def preprocess_lstm(features):
    
    print("+++ Preprocessing data... +++")
    try:
        violent_features = features[0:30*math.floor(violent_frames.shape[0]/30)]
    except:
        pass
    try:
        non_violent_features = features[violent_frames.shape[0]:violent_frames.shape[0]+30*math.floor(non_violent_frames.shape[0]/30)]
    except:
        pass

    print("Violent features: ", violent_features.shape)
    print("Non Violent features: ", non_violent_features.shape)


    violent_vid,non_violent_vid = create_seq(violent_features, non_violent_features)
    violent_y, non_violent_y = np.zeros(len(violent_vid)), np.ones(len(non_violent_vid))

    print("Violent Video Seq: ", violent_vid.shape,"Non_violent video Seq: ", non_violent_vid.shape)
    print("Violent Label: ", violent_y.shape, "Non_violent Label: ", non_violent_y.shape)

    try:
        test_x = np.vstack((violent_vid,non_violent_vid))
        test_y = np.append(violent_y, non_violent_y)
    except:
        test_x = non_violent_vid
        test_y = non_violent_y

    print("Total data: ", test_x.shape)
    print("Total target: ", test_y.shape)

    test_x = np.reshape(test_x, (test_x.shape[0],test_x.shape[1],np.prod(test_x.shape[2:])))
    print("(LSTM) After Rehshape: ", test_x.shape)
    
    return test_x,test_y

In [107]:
def feature_extract(violent_frames, non_violent_frames, model):
    
    print("+++ Extracting feature... +++")
    
    violent_frames = np.asarray(violent_frames)
    non_violent_frames = np.asarray(non_violent_frames)

    print ("Before Feature extraction: ")
    print(violent_frames.shape,non_violent_frames.shape)
    all_data = np.vstack((violent_frames,non_violent_frames))
    print("Adding all data: ", all_data.shape)

    #creates feature descriptors
    all_data = all_data.astype('float64')
    desc = preprocess_input(all_data)
    if(model == 'resnet50'):
        loaded_model = resnet50.ResNet50(input_shape=(224,224,3), include_top=False)
    elif(model == 'vgg19'):
        loaded_model = VGG19(input_shape=(224,224,3), include_top=False)
    elif(model == 'vgg16'):
        loaded_model = VGG16(input_shape=(224,224,3), include_top=False)
    else:
        print("Please give model name - 'resnet50', 'vgg19', 'vgg16'")
        
    loaded_model = Model(loaded_model.input,loaded_model.output)
    features = loaded_model.predict(desc,batch_size=10,verbose=1)

    print ("After Feature extraction: ", features.shape)
    
    return features

In [108]:
print("+++ Extracting feature... +++")


violent_frames = np.asarray(violent_frames)
non_violent_frames = np.asarray(non_violent_frames)

#violent_frames = violent_frames.reshape((0,0,0,0))


# try:
#     print(violent_frames.shape[3])
# except:
#     violent_frames = violent_frames.reshape(0,0,0,0)
    
# try:
#     print(non_violent_frames.shape[3])
# except:
#     non_violent_frames = non_violent_frames.reshape(0,0,0,0)
    
print ("Before Feature extraction: ")
print(violent_frames.shape,non_violent_frames.shape)


#all_data = np.vstack((violent_frames,non_violent_frames))
try:
    all_data = np.vstack((violent_frames,non_violent_frames))
except:
    if non_violent_frames.shape[0]!=0:
        all_data = non_violent_frames
    else:
        all_data = violent_frames
print("Adding all data: ", all_data.shape)

#creates feature descriptors
all_data = all_data.astype('float64')
desc = preprocess_input(all_data)
model = 'resnet50'
if(model == 'resnet50'):
    loaded_model = resnet50.ResNet50(input_shape=(224,224,3), include_top=False)
elif(model == 'vgg19'):
    loaded_model = VGG19(input_shape=(224,224,3), include_top=False)
elif(model == 'vgg16'):
    loaded_model = VGG16(input_shape=(224,224,3), include_top=False)
else:
    print("Please give model name - 'resnet50', 'vgg19', 'vgg16'")

+++ Extracting feature... +++
Before Feature extraction: 
(3587, 224, 224, 3) (2660, 224, 224, 3)
Adding all data:  (6247, 224, 224, 3)


In [109]:

loaded_model = Model(loaded_model.input,loaded_model.output)
features = loaded_model.predict(desc,batch_size=100,verbose=1)

print ("After Feature extraction: ", features.shape)

63/63 [==============================] - 1335s 21s/step
After Feature extraction:  (6247, 7, 7, 2048)


In [110]:
import math
test_x, test_y = preprocess_lstm(features)

+++ Preprocessing data... +++
Violent features:  (3570, 7, 7, 2048)
Non Violent features:  (2640, 7, 7, 2048)
+++ Creating Sequence... +++
Violent Video Seq:  (119, 30, 7, 7, 2048) Non_violent video Seq:  (88, 30, 7, 7, 2048)
Violent Label:  (119,) Non_violent Label:  (88,)
Total data:  (207, 30, 7, 7, 2048)
Total target:  (207,)
(LSTM) After Rehshape:  (207, 30, 100352)


In [111]:
model = Sequential()
model.add(LSTM(50, input_shape=(test_x.shape[1],test_x.shape[2]), return_sequences=False, kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1,activation='sigmoid'))
model.load_weights('resnet_LSTM_v1.h5')
model.summary()
optimizer = optimizers.Adam(lr=0.001,decay=0.004)
model.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_14 (LSTM)               (None, 50)                20080600  
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 51        
Total params: 20,080,651
Trainable params: 20,080,651
Non-trainable params: 0
_________________________________________________________________


In [112]:
result(test_x,test_y)

+++ Generating result... +++
7/7 [==============================] - 30s 4s/step - loss: 0.5143 - accuracy: 0.8116
-----------------------------
Test loss: 0.5142815709114075
Test accuracy: 0.8115941882133484
-----------------------------
Classification report
-----------------------------
              precision    recall  f1-score   support

         0.0       0.97      0.70      0.81       119
         1.0       0.70      0.97      0.81        88

    accuracy                           0.81       207
   macro avg       0.83      0.83      0.81       207
weighted avg       0.85      0.81      0.81       207

-----------------------------
Confusion Matrix
-----------------------------
[[83 36]
 [ 3 85]]


In [113]:
model = Sequential()
model.add(LSTM(50, input_shape=(test_x.shape[1],test_x.shape[2]), return_sequences=False, kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1,activation='sigmoid'))
model.load_weights('resnet_LSTM_v3.h5')
model.summary()
optimizer = optimizers.Adam(lr=0.001,decay=0.004)
model.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (None, 50)                20080600  
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 51        
Total params: 20,080,651
Trainable params: 20,080,651
Non-trainable params: 0
_________________________________________________________________


In [114]:
result(test_x,test_y)

+++ Generating result... +++
7/7 [==============================] - 30s 4s/step - loss: 0.9266 - accuracy: 0.6425
-----------------------------
Test loss: 0.9266251921653748
Test accuracy: 0.6425120830535889
-----------------------------
Classification report
-----------------------------
              precision    recall  f1-score   support

         0.0       0.89      0.43      0.58       119
         1.0       0.55      0.93      0.69        88

    accuracy                           0.64       207
   macro avg       0.72      0.68      0.63       207
weighted avg       0.75      0.64      0.63       207

-----------------------------
Confusion Matrix
-----------------------------
[[51 68]
 [ 6 82]]


In [80]:
model = Sequential()
model.add(LSTM(100, input_shape=(test_x.shape[1],test_x.shape[2]), return_sequences=False, kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1,activation='sigmoid'))
model.load_weights('resnet_LSTM_v2.h5')
model.summary()
optimizer = optimizers.Adam(lr=0.001,decay=0.004)
model.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 100)               40181200  
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 101       
Total params: 40,181,301
Trainable params: 40,181,301
Non-trainable params: 0
_________________________________________________________________


In [81]:
result(test_x,test_y)

+++ Generating result... +++
2/2 [==============================] - 0s 202ms/step - loss: 0.2099 - accuracy: 0.9048
-----------------------------
Test loss: 0.20985019207000732
Test accuracy: 0.9047619104385376
-----------------------------
Classification report
-----------------------------
              precision    recall  f1-score   support

         0.0       0.90      0.90      0.90        21
         1.0       0.90      0.90      0.90        21

    accuracy                           0.90        42
   macro avg       0.90      0.90      0.90        42
weighted avg       0.90      0.90      0.90        42

-----------------------------
Confusion Matrix
-----------------------------
[[19  2]
 [ 2 19]]


In [82]:
model = Sequential()
model.add(LSTM(50, input_shape=(test_x.shape[1],test_x.shape[2]), return_sequences=False, kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1,activation='sigmoid'))
model.load_weights('resnet_LSTM_v1.h5')
model.summary()
optimizer = optimizers.Adam(lr=0.001,decay=0.004)
model.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 50)                20080600  
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 51        
Total params: 20,080,651
Trainable params: 20,080,651
Non-trainable params: 0
_________________________________________________________________


In [83]:
result(test_x,test_y)

+++ Generating result... +++
2/2 [==============================] - 0s 137ms/step - loss: 0.1404 - accuracy: 1.0000
-----------------------------
Test loss: 0.14040739834308624
Test accuracy: 1.0
-----------------------------
Classification report
-----------------------------
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        21
         1.0       1.00      1.00      1.00        21

    accuracy                           1.00        42
   macro avg       1.00      1.00      1.00        42
weighted avg       1.00      1.00      1.00        42

-----------------------------
Confusion Matrix
-----------------------------
[[21  0]
 [ 0 21]]


In [88]:
model = Sequential()
model.add(LSTM(100, input_shape=(test_x.shape[1],test_x.shape[2]), return_sequences=False, kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1,activation='sigmoid'))
model.load_weights('resnet_LSTM_v2.h5')
model.summary()
optimizer = optimizers.Adam(lr=0.001,decay=0.004)
model.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 100)               40181200  
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 101       
Total params: 40,181,301
Trainable params: 40,181,301
Non-trainable params: 0
_________________________________________________________________


In [89]:
result(test_x,test_y)

+++ Generating result... +++
19/19 [==============================] - 38s 2s/step - loss: 1.8649 - accuracy: 0.4959
-----------------------------
Test loss: 1.8648954629898071
Test accuracy: 0.49585404992103577
-----------------------------
Classification report
-----------------------------
              precision    recall  f1-score   support

         0.0       0.10      0.94      0.18        36
         1.0       0.99      0.47      0.64       567

    accuracy                           0.50       603
   macro avg       0.55      0.71      0.41       603
weighted avg       0.94      0.50      0.61       603

-----------------------------
Confusion Matrix
-----------------------------
[[ 34   2]
 [302 265]]


In [90]:
model = Sequential()
model.add(LSTM(50, input_shape=(test_x.shape[1],test_x.shape[2]), return_sequences=False, kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1,activation='sigmoid'))
model.load_weights('resnet_LSTM_v1.h5')
model.summary()
optimizer = optimizers.Adam(lr=0.001,decay=0.004)
model.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 50)                20080600  
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 51        
Total params: 20,080,651
Trainable params: 20,080,651
Non-trainable params: 0
_________________________________________________________________


In [91]:
result(test_x,test_y)

+++ Generating result... +++
19/19 [==============================] - 27s 1s/step - loss: 0.8191 - accuracy: 0.7081
-----------------------------
Test loss: 0.8190941214561462
Test accuracy: 0.7081260085105896
-----------------------------
Classification report
-----------------------------
              precision    recall  f1-score   support

         0.0       0.17      1.00      0.29        36
         1.0       1.00      0.69      0.82       567

    accuracy                           0.71       603
   macro avg       0.58      0.84      0.55       603
weighted avg       0.95      0.71      0.78       603

-----------------------------
Confusion Matrix
-----------------------------
[[ 36   0]
 [176 391]]


In [92]:
model = Sequential()
model.add(LSTM(50, input_shape=(test_x.shape[1],test_x.shape[2]), return_sequences=False, kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1,activation='sigmoid'))
model.load_weights('resnet_LSTM_v3.h5')
model.summary()
optimizer = optimizers.Adam(lr=0.001,decay=0.004)
model.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 50)                20080600  
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 51        
Total params: 20,080,651
Trainable params: 20,080,651
Non-trainable params: 0
_________________________________________________________________


In [93]:
result(test_x,test_y)

+++ Generating result... +++
19/19 [==============================] - 29s 2s/step - loss: 1.0589 - accuracy: 0.6153
-----------------------------
Test loss: 1.058946132659912
Test accuracy: 0.6152570247650146
-----------------------------
Classification report
-----------------------------
              precision    recall  f1-score   support

         0.0       0.12      0.89      0.22        36
         1.0       0.99      0.60      0.75       567

    accuracy                           0.62       603
   macro avg       0.56      0.74      0.48       603
weighted avg       0.94      0.62      0.71       603

-----------------------------
Confusion Matrix
-----------------------------
[[ 32   4]
 [228 339]]


In [31]:
model = Sequential()
model.add(LSTM(50, input_shape=(test_x.shape[1],test_x.shape[2]), return_sequences=False, kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1,activation='sigmoid'))
model.load_weights('resnet_LSTM.h5')
model.summary()
optimizer = optimizers.Adam(lr=0.001,decay=0.004)
model.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 50)                20080600  
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 51        
Total params: 20,080,651
Trainable params: 20,080,651
Non-trainable params: 0
_________________________________________________________________


In [32]:
result(test_x,test_y)

+++ Generating result... +++
25/25 [==============================] - 7s 281ms/step - loss: 0.6929 - accuracy: 0.7235
-----------------------------
Test loss: 0.6929287910461426
Test accuracy: 0.7234848737716675
-----------------------------
Classification report
-----------------------------
              precision    recall  f1-score   support

         0.0       0.93      0.73      0.81       660
         1.0       0.34      0.71      0.46       132

    accuracy                           0.72       792
   macro avg       0.63      0.72      0.64       792
weighted avg       0.83      0.72      0.76       792

-----------------------------
Confusion Matrix
-----------------------------
[[479 181]
 [ 38  94]]


In [28]:
result(test_x,test_y)

+++ Generating result... +++
25/25 [==============================] - 8s 313ms/step - loss: 0.6929 - accuracy: 0.7235
-----------------------------
Test loss: 0.6929287910461426
Test accuracy: 0.7234848737716675
-----------------------------
Classification report
-----------------------------
              precision    recall  f1-score   support

         0.0       0.93      0.73      0.81       660
         1.0       0.34      0.71      0.46       132

    accuracy                           0.72       792
   macro avg       0.63      0.72      0.64       792
weighted avg       0.83      0.72      0.76       792

-----------------------------
Confusion Matrix
-----------------------------
[[479 181]
 [ 38  94]]


In [12]:
result(test_x,test_y)

+++ Generating result... +++
5/5 [==============================] - 1s 134ms/step - loss: 0.7443 - accuracy: 0.7121
-----------------------------
Test loss: 0.7442625761032104
Test accuracy: 0.7121211886405945
-----------------------------
Classification report
-----------------------------
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.71      0.83       132

    accuracy                           0.71       132
   macro avg       0.50      0.36      0.42       132
weighted avg       1.00      0.71      0.83       132

-----------------------------
Confusion Matrix
-----------------------------
[[ 0  0]
 [38 94]]


c:\users\administrator\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
